In [ ]:
import numpy as np
import pandas as pd

from src.main import generate_blackout_events, compute_blackout_kpis

average_frequency = 45
std_frequency = 0.1
average_duration = 3
std_duration = 0.1

evaluated_days = 3
date_time_index = pd.date_range(start="2024-01-01",freq="H", periods=evaluated_days * 24)


grid_availability, blackout_events_duration, overlapping_blackouts, blackout_count = generate_blackout_events(average_frequency, average_duration, date_time_index)
print(overlapping_blackouts, blackout_count)

compute_blackout_kpis(grid_availability, blackout_events_duration, overlapping_blackouts, blackout_count)
